In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.neighbors import KDTree
import time
import argparse
import json

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


can I start from specific timestamps ?

what happens for skip_frames=0 ?

In [2]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width=200):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        if not ret:
            break
        # skip every "skip_frame"
        if vid_length>=start_frame:
            # resize the video to a different resolution
            frame=cv2.resize(frame,resolution)
            frames.append(frame) #add the individual frames to a list
        if vid_length==end_frame:
            break
        vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    return frames

Dominant Color Extractor

- add better comments

In [3]:
def extract_dominant_color(frame_list):
    print(str(len(frame_list))+' frames to process.')
    start=time.time()
    rgb_to_color=fn_rgb_to_color() #get the color dict 
    bins={} #a dict with an entry for each for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
        
    rgb_list=[] #create a list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    i = 0

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')  
    for image in frame_list: #traverse the video
        #flatten the image to 1d 
        img = image.reshape((image.shape[0] * image.shape[1], 3))     
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[rgb_to_color[rgb_list[nn[0]]]]+=1
#        for pixel in img: # do nearest neighbour search on every pixel every color in the list
#            bin_aux=[]
#            #get the euclidean distance between the colors and the current pixel
#            for rgb in rgb_list:
#                bin_aux.append(euclidean(pixel,rgb))
#            # get the index of the color,which has the smallest distance, in rgb_list
#            min_pos = np.argmin(bin_aux)
#            #increment the respective color 
#            bins[rgb_to_color[rgb_list[min_pos]]]+=1
        i+=1
        end=time.time()
        print('Finished '+str(i)+',time: '+str(end-start))
        norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1]#normalize the bins
        bins_norm={k:v/norm_factor for k,v in bins.items()}
    return bins_norm

In [4]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe, sorted descending by count
    bins_sorted=sorted(zip(list(bins.values()),list(bins.keys())),reverse=True)
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
    bin_threshold=bin_threshold/100 #scale the percentage to 0-1
    df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df.head(colors_to_return)#return the color_return highest bins, default 5, if less bins then
                                    #color_return are there return all

In [39]:
def fn_rgb_to_color(*path):
    if not ('no'):
        path=str(path)[2:-3] #to get rid of the of the *args things
        rgb_to_color = {}
        with open(path) as f:
            for line in f:
                #split lines at "::
                color, rgb = line.strip().split(':')
                #strip the rgb-string of the parenthesis, split it up a the commas,
                #cast them to int and put them into a tuples
                rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                rgb_to_color[rgb_value] = color
    else:
        print('dfd')
        colors={'darkred':(139,0,0),
        'firebrick':(178,34,34),
        'crimson':(220,20,60),
        'red':(255,0,0),
        'tomato':(255,99,71),
        'salmon':(250,128,114),
        'darkorange':(255,140,0),
        'gold':(255,215,0),
        'darkkhaki':(189,183,107),
        'yellow':(255,255,0),
        'darkolivegreen':(85,107,47),
        'olivedrab':(107,142,35),
        'greenyellow':(173,255,47),
        'darkgreen':(0,100,0),
        'aquamarine':(127,255,212),
        'steelblue':(70,130,180),
        'skyblue':(135,206,235),
        'darkblue':(0,0,139),
        'blue':(0,0,255),
        'royalblue':(65,105,225),
        'purple':(128,0,128),
        'violet':(238,130,238),
        'deeppink':(255,20,147),
        'pink':(255,192,203),
        'antiquewhite':(250,235,215),
        'saddlebrown':(139,69,19),
        'sandybrown':(244,164,96),
        'ivory':(255,255,240),
        'dimgrey':(105,105,105),
        'grey':(28,128,128),
        'silver':(192,192,192),
        'lightgrey':(211,211,211),
        'black':(0,0,0),
        'white':(255,255,255),
        'darkcyan':(0,139,139),
        'cyan':(0,255,255),
        'green':(0,128,0),
        'khaki':(240,230,140),
        'goldenrod':(218,165,32),
        'orange':(255,165,0),
        'coral':(255,127,80),
        'magenta':(255,0,255),
        'wheat':(245,222,179),
        'skin':(255,224,189),
        'purple4':(147,112,219)}
        rgb_to_color={}
        for color in colors:
            rgb_to_color[colors[color]]=color
        #purple4 is median purple
        #skin is caucasian
    return rgb_to_color

In [6]:
#read the file with colors and there rgb-values into a dictionary
def read_color_from_file(path):
    rgb_to_color = {}
    with open(path) as f:
        for line in f:
            #split lines at "::
            color, rgb = line.strip().split(':')
            #strip the rgb-string of the parenthesis, split it up a the commas,
            #cast them to int and put them into a tuples
            rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
            rgb_to_color[rgb_value] = color
    return rgb_to_color

In [31]:
black=np.array([[[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]],
                [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]],
                [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]])
red=np.array([[[255,0,0],[255,0,0],[255,0,0],[255,0,0],[255,0,0]],
             [[255,0,0],[255,0,0],[255,0,0],[255,0,0],[255,0,0]],
              [[255,0,0],[255,0,0],[255,0,0],[255,0,0],[255,0,0]]])
green=np.array([[[0,255,0],[0,255,0],[0,255,0],[0,255,0],[0,255,0]],
               [[0,255,0],[0,255,0],[0,255,0],[0,255,0],[0,255,0]],
               [[0,255,0],[0,255,0],[0,255,0],[0,255,0],[0,255,0]]])
blue=np.array([[[0,0,255],[0,0,255],[0,0,255],[0,0,255],[0,0,255]],
              [[0,0,255],[0,0,255],[0,0,255],[0,0,255],[0,0,255]],
              [[0,0,255],[0,0,255],[0,0,255],[0,0,255],[0,0,255]]])
white=np.array([[[255,255,255],[255,255,255],[255,255,255],[255,255,255],[255,255,255]],
               [[255,255,255],[255,255,255],[255,255,255],[255,255,255],[255,255,255]],
               [[255,255,255],[255,255,255],[255,255,255],[255,255,255],[255,255,255]]])

In [41]:
a=[black,black,black]
print(np.shape(a))
bins_to_df(extract_dominant_color(a))

(3, 3, 5, 3)
3 frames to process.
dfd
Finished 1,time: 0.0006577968597412109
Finished 2,time: 0.0011055469512939453
Finished 3,time: 0.001505136489868164


,count
color,
black,1.0


In [33]:
b = read_video_segments('videos/red.mp4',0,1,5)
print(np.shape(b))


(2, 3, 5, 3)
